In [1]:
import sqlite3
import ttkbootstrap as tb
from ttkbootstrap.constants import *
from tkinter import messagebox

class CRUDApp(tb.Window):
    def __init__(self):
        super().__init__(themename="darkly")
        self.title("CRUD App - Gestión de Usuarios")
        self.geometry("540x400")
        self.resizable(False, False)

        # Conectar a la base de datos
        self.conn = sqlite3.connect(":memory:") # Base de datos en memoria para pruebas
        self.cursor = self.conn.cursor()
        self.cursor.execute("""
            CREATE TABLE IF NOT EXISTS usuarios (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                nombre TEXT NOT NULL,
                correo TEXT NOT NULL
            )
        """)
        self.conn.commit()

        self.create_widgets()
        self.read_users()

    def create_widgets(self):
        tb.Label(self, text="Nombre:").grid(row=0, column=0, padx=10, pady=(20,5), sticky="e")
        self.nombre_entry = tb.Entry(self, width=40)
        self.nombre_entry.grid(row=0, column=1, padx=10, pady=(20,5), sticky="ew")

        tb.Label(self, text="Correo:").grid(row=1, column=0, padx=10, pady=5, sticky="e")
        self.correo_entry = tb.Entry(self, width=40)
        self.correo_entry.grid(row=1, column=1, padx=10, pady=5, sticky="ew")

        tb.Button(self, text="Create", bootstyle="success", command=self.create_user).grid(row=2, column=0, padx=10, pady=10)
        tb.Button(self, text="Update", bootstyle="warning", command=self.update_user).grid(row=2, column=1, padx=10, pady=10, sticky="w")
        tb.Button(self, text="Delete", bootstyle="danger", command=self.delete_user).grid(row=2, column=1, padx=10, pady=10, sticky="e")

        self.tree = tb.Treeview(self, columns=("id", "nombre", "correo"), show="headings", bootstyle="dark")

        self.tree.heading("id", text="ID")
        self.tree.heading("nombre", text="Nombre")
        self.tree.heading("correo", text="Correo")

        self.tree.column("id", width=50, anchor="center")       
        self.tree.column("nombre", width=200, anchor="w")
        self.tree.column("correo", width=250, anchor="w")

        self.tree.grid(row=3, column=0, columnspan=2, padx=10, pady=10, sticky="nsew")

        self.tree.bind("<<TreeviewSelect>>", self.fill_entries_from_selection)

    def create_user(self):
        nombre = self.nombre_entry.get()
        correo = self.correo_entry.get()
        if nombre and correo:
            self.cursor.execute("INSERT INTO usuarios (nombre, correo) VALUES (?, ?)", (nombre, correo))
            self.conn.commit()
            self.clear_entries()
            self.read_users()
        else:
            messagebox.showwarning("Datos incompletos", "Por favor, llena todos los campos.")

    def read_users(self):
        for row in self.tree.get_children():
            self.tree.delete(row)
        self.cursor.execute("SELECT * FROM usuarios")
        for user in self.cursor.fetchall():
            self.tree.insert("", "end", values=user)

    def update_user(self):
        selected = self.tree.focus()
        if selected:
            user_id = self.tree.item(selected)["values"][0]
            nombre = self.nombre_entry.get()
            correo = self.correo_entry.get()
            self.cursor.execute("UPDATE usuarios SET nombre=?, correo=? WHERE id=?", (nombre, correo, user_id))
            self.conn.commit()
            self.clear_entries()
            self.read_users()
        else:
            messagebox.showinfo("Selecciona un registro", "Debes seleccionar un usuario para actualizar.")

    def delete_user(self):
        selected = self.tree.focus()
        if selected:
            user_id = self.tree.item(selected)["values"][0]
            self.cursor.execute("DELETE FROM usuarios WHERE id=?", (user_id,))
            self.conn.commit()
            self.clear_entries()
            self.read_users()
        else:
            messagebox.showinfo("Selecciona un registro", "Debes seleccionar un usuario para eliminar.")

    def fill_entries_from_selection(self, event):
        selected = self.tree.focus()
        if selected:
            values = self.tree.item(selected)["values"]
            self.nombre_entry.delete(0, "end")
            self.nombre_entry.insert(0, values[1])
            self.correo_entry.delete(0, "end")
            self.correo_entry.insert(0, values[2])

    def clear_entries(self):
        self.nombre_entry.delete(0, "end")
        self.correo_entry.delete(0, "end")

if __name__ == "__main__":
    app = CRUDApp()
    app.mainloop()
